In [1]:
import numpy as np
from methods.regression_methods import NGGP, get_transforms
from run_regression import *
from training.configs import Config as config
from models import backbone
import torch
from argparse import Namespace
from data.neural_loader import *
from torch import tensor
import gpytorch
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'nggp_lib'

In [ ]:
save_path_th = '/Users/jessegill/Desktop/nggp/nggp_lib/save/nggp_rbf_5e3/checkpoints/neural/MLP2_NGGP_model.th'
data = torch.load(save_path_th)
params = Namespace(seed=1, model='MLP2', method='NGGP', dataset='neural', update_batch_size=5, meta_batch_size=5, output_dim=40, multidimensional_amp=False, multidimensional_phase=False, noise='gaussian', kernel_type='rbf', save_dir='./save/nggp_rbf_5e3', num_tasks=1, multi_type=3, method_lr=0.001, feature_extractor_lr=0.001, cnf_lr=0.001, all_lr=0.005, neptune=False, use_conditional=False, context_type='backbone', layer_type='concatsquash', dims='32-32', num_blocks=2, time_length=0.5, train_T=False, add_noise=False, divergence_fn='brute_force', nonlinearity='tanh', solver='dopri5', atol=1e-05, rtol=1e-05, step_size=None, test_solver=None, test_atol=None, test_rtol=None, residual=False, rademacher=False, spectral_norm=False, batch_norm=False, bn_lag=0, l1int=None, l2int=None, dl2int=None, JFrobint=None, JdiagFrobint=None, JoffdiagFrobint=None, start_epoch=0, stop_epoch=100, test=False, n_support=5, n_test_epochs=10, out_of_range=False, device='cuda')

In [ ]:
save_path_th = '/Users/jessegill/Desktop/nggp/nggp_lib/save/sines_1e6/checkpoints/sines/MLP2_NGGP_model.th'
params = Namespace(seed=1, model='MLP2', method='NGGP', dataset='sines', update_batch_size=5, meta_batch_size=5, output_dim=40, multidimensional_amp=False, multidimensional_phase=False, noise='gaussian', kernel_type='rbf', save_dir='./save/sines_1e6', num_tasks=1, multi_type=3, method_lr=0.001, feature_extractor_lr=0.001, cnf_lr=0.001, all_lr=1e-06, neptune=False, use_conditional=True, context_type='backbone', layer_type='concatsquash', dims='32-32', num_blocks=2, time_length=0.5, train_T=False, add_noise=False, divergence_fn='brute_force', nonlinearity='tanh', solver='dopri5', atol=1e-05, rtol=1e-05, step_size=None, test_solver=None, test_atol=None, test_rtol=None, residual=False, rademacher=False, spectral_norm=False, batch_norm=False, bn_lag=0, l1int=None, l2int=None, dl2int=None, JFrobint=None, JdiagFrobint=None, JoffdiagFrobint=None, start_epoch=0, stop_epoch=100, test=False, n_support=5, n_test_epochs=10, out_of_range=False, device='cuda')

In [ ]:
run = True
setup_seed(params)
config = Config(params)
checkpoint_dir, save_path = setup_checkpoint_dir(params)

results_logger = ResultsLogger(params)


device = 'cpu'
logging.info('Device: {}'.format(device))

bb = setup_backbone(device, params)
nggp_model = setup_model(bb, config, device, params)
optimizer = setup_optimizer(nggp_model, params)

if params.test and run:
    test(nggp_model, params, save_path, results_logger)
elif run:
    train(nggp_model, optimizer, params, save_path, results_logger)
setup_checkpoint_dir(params)
nggp_model.load_checkpoint(save_path_th, device)

In [ ]:
neural_loader = NeuralDatasetLoader()
batch = neural_loader.get_batch()
list(zip(batch[0],batch[1]))
neural_loader.generate_datastack()

In [ ]:
all_batches = []
all_batch_labels = []
while True:  # or some condition to end the loop
    batch, batch_labels = neural_loader.get_batch()
    if batch is None or batch_labels is None:
        break  # Exit loop if there's no more data



    all_batches.append(batch)
    all_batch_labels.append(batch_labels)

# Now, all_batches and all_batch_labels are lists of tensors, each representing a batch
# Example: iterate through each batch
dataloader = zip(all_batches,all_batch_labels)
print(all_batch_labels)


In [ ]:
sample_fn, _ = get_transforms(nggp_model.cnf, nggp_model.use_conditional)

In [ ]:
with open("/Users/jessegill/Desktop/nggp/data/ST260_Day1.pkl", 'rb') as f:
    data = pickle.load(f)

data['time']

In [ ]:
test_x = all_batches[0]
test_labels = all_batch_labels[0]

test_x = torch.linspace(0, 15, 3).unsqueeze(0).T

test_x = data['time'][:40]
test_x = torch.tensor(test_x, dtype=torch.float32).unsqueeze(1)  # reshape to (N, 1)

In [ ]:
test_x = torch.tensor(np.linspace(0,10,40), dtype=torch.float32).unsqueeze(1)
nggp_model.model.eval()  # Set the model to evaluation mode


print("Original shape of test_x:", test_x.shape)

# If your model expects an input shape of [batch_size, N], reshape test_x accordingly
# For example, if N is 40:
#test_x = test_x.view(-1, 40)  # Reshape to [batch_size, 40]

print("Reshaped test_x:", test_x.shape)

# Continue with the model prediction
z = nggp_model.feature_extractor(test_x)

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = nggp_model.model(z)
    means = sample_fn(predictions.mean.unsqueeze(1))


print(means)
# Plot the mean predictions
plt.figure(figsize=(10, 6))
plt.plot(test_x, means)
plt.title("Mean Prediction of Gaussian Process")
plt.xlabel("Input")
plt.ylabel("Mean Prediction")
plt.show()
